# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [36]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [37]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nemuro,43.3236,145.5750,264.25,75,51,1.61,JP,1675459617
1,1,san patricio,28.0170,-97.5169,288.80,55,5,3.40,US,1675459617
2,2,busselton,-33.6500,115.3333,292.51,60,0,5.72,AU,1675459618
3,3,jamestown,42.0970,-79.2353,260.77,84,100,9.26,US,1675459385
4,4,jijiga,9.3500,42.8000,289.90,65,8,4.81,ET,1675459618


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [70]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Lat"]>= -30) & (city_data_df["Lat"]<= 30)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,jijiga,9.3500,42.8000,289.90,65,8,4.81,ET,1675459618
8,8,luanda,-8.8368,13.2343,300.15,78,40,5.14,AO,1675459620
13,13,wao,7.6833,124.6833,289.44,71,70,1.24,PH,1675459623
20,20,lazaro cardenas,17.9583,-102.2000,302.03,48,0,4.58,MX,1675459629
26,26,barra do garcas,-15.8900,-52.2567,294.47,100,75,0.00,BR,1675459636
29,29,mlonggo,-6.5333,110.7000,299.47,85,100,5.35,ID,1675459639
30,30,puro,12.4841,123.3827,298.00,83,99,5.86,PH,1675459639
32,32,daru,-9.0763,143.2092,298.25,88,100,4.67,PG,1675459641
35,35,namibe,-15.1961,12.1522,295.70,75,94,2.09,AO,1675459643
52,52,yangambi,0.8102,24.4336,297.49,64,60,0.74,CD,1675459655


### Step 3: Create a new DataFrame called `hotel_df`.

In [71]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City','Country',"Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""
# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
4,jijiga,ET,9.3500,42.8000,65,
8,luanda,AO,-8.8368,13.2343,78,
13,wao,PH,7.6833,124.6833,71,
20,lazaro cardenas,MX,17.9583,-102.2000,48,
26,barra do garcas,BR,-15.8900,-52.2567,100,
29,mlonggo,ID,-6.5333,110.7000,85,
30,puro,PH,12.4841,123.3827,83,
32,daru,PG,-9.0763,143.2092,88,
35,namibe,AO,-15.1961,12.1522,75,
52,yangambi,CD,0.8102,24.4336,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [72]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{row['Lat']},{row['Lng']},{radius}"
    params["bias"] = f"proximity:{row['Lat']},{row['Lng']}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jijiga - nearest hotel: Bergerie laggiarelli
luanda - nearest hotel: No hotel found
wao - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: No hotel found
barra do garcas - nearest hotel: No hotel found
mlonggo - nearest hotel: No hotel found
puro - nearest hotel: No hotel found
daru - nearest hotel: No hotel found
namibe - nearest hotel: No hotel found
yangambi - nearest hotel: No hotel found
luena - nearest hotel: No hotel found
atuona - nearest hotel: No hotel found
butaritari - nearest hotel: No hotel found
kavaratti - nearest hotel: No hotel found
gigmoto - nearest hotel: No hotel found
hithadhoo - nearest hotel: No hotel found
airai - nearest hotel: No hotel found
hilo - nearest hotel: No hotel found
santa cruz - nearest hotel: No hotel found
bengkulu - nearest hotel: No hotel found
amapa - nearest hotel: No hotel found
isangel - nearest hotel: No hotel found
dingle - nearest hotel: No hotel found
puerto escondido - nearest hotel: No hotel found

,City,Country,Lat,Lng,Humidity,Hotel Name
4,jijiga,ET,9.3500,42.8000,65,Bergerie laggiarelli
8,luanda,AO,-8.8368,13.2343,78,No hotel found
13,wao,PH,7.6833,124.6833,71,No hotel found
20,lazaro cardenas,MX,17.9583,-102.2000,48,No hotel found
26,barra do garcas,BR,-15.8900,-52.2567,100,No hotel found
...,...,...,...,...,...,...
558,mitsamiouli,KM,-11.3847,43.2844,79,No hotel found
559,sinnamary,GF,5.3833,-52.9500,85,No hotel found
560,mumford,GH,5.2618,-0.7590,87,No hotel found
562,luang prabang,LA,19.8856,102.1347,87,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [73]:
%%capture --no-display

# Configure the map plot
hotel_df = hotel_df.loc[hotel_df['Hotel Name']!= "No hotel found"]

hot_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    hover_cols=["Hotel Name"],
    color = "City"
)

# Display the map
hot_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)